### Exercicio

#### Montem uma classificacao utilizando a base da Veltec ou Senai
#### Monte testes de avaliacao de diferentes classificadores considerando:
* Busca por hiperparametros 
    * Considere testar parametros de regularizacao
* Busca por features
* Utilize um metodo de validacao cruzada

In [301]:
# Manipulacao de dados e operacoes
import pandas as pd 
import numpy as np 
from numpy import median
from itertools import combinations
from sklearn.model_selection import train_test_split
import math

# Visualizacao
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()

# Estatistica
from statistics import mean 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import classification_report    # output: accuracy, f1-score, recall e precision
from sklearn.metrics import confusion_matrix

# Modelos de regressao
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.tree import DecisionTreeRegressor  
from sklearn.neighbors import KNeighborsRegressor

# Modelos de Classificação
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Outras bibliotecas de modelos
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, LeaveOneOut
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.externals import joblib

# Pre-processamento
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Outros
from datetime import datetime, timedelta
from scipy.special import expit
from scipy.io import arff
from sklearn.datasets import fetch_olivetti_faces, load_iris
from sklearn.feature_selection import SelectKBest, chi2

In [302]:
# Lendo a base de dados
senai = pd.read_csv("senai_inep2.csv")

In [303]:
# Tamanho do dataframe
senai.shape

(3309, 22)

In [304]:
#duplicados = senai[senai['ID_ALUNO'].duplicated() == True]['ID_ALUNO']

# Separando as bases em X e y
---

In [305]:
# Codificando colunas categóricas
le = LabelEncoder()
senai['NO_IES'] = le.fit_transform(senai['NO_IES'])
senai['NO_CURSO'] = le.fit_transform(senai['NO_IES'])
senai['ID_ALUNO'] = le.fit_transform(senai['ID_ALUNO'])
senai['DT_INGRESSO_CURSO'] = le.fit_transform(senai['DT_INGRESSO_CURSO'])

# Saída a ser predita "y"
X = senai.drop(['TP_SITUACAO'], axis=1)
y = senai[['TP_SITUACAO']]

In [306]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

### Comparando scores dos modelos:

*Decision Tree* apresenta maior score.

In [307]:
# Regressao Logistica
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train.values.ravel())
y_pred = log_reg.predict(X_test)
print("Logistic Regression score:", log_reg.score(X_test, y_test))

# KNN
knn = KNeighborsClassifier()
knn.fit(X_train, y_train.values.ravel())
y_pred = knn.predict(X_test)
print("KNN score:", knn.score(X_test, y_test))

# Arvore de Decisao
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("Tree score:", tree.score(X_test, y_test))

# Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train.values.ravel())
y_pred = nb.predict(X_test)
print("Naive Bayes score:", nb.score(X_test, y_test))

# Ridge
ridge = RidgeClassifier()
ridge.fit(X_train, y_train.values.ravel())
y_pred = ridge.predict(X_test)
print("Ridge score:", ridge.score(X_test, y_test))

# SGD
sgd = SGDClassifier()
sgd.fit(X_train, y_train.values.ravel())
y_pred = sgd.predict(X_test)
print("SGD score:", sgd.score(X_test, y_test))

Logistic Regression score: 0.8873239436619719
KNN score: 0.8812877263581489
Tree score: 0.8853118712273642
Naive Bayes score: 0.8873239436619719
Ridge score: 0.9114688128772636
SGD score: 0.8873239436619719


# Fazendo a mesma coisa, mas agora com Pipeline
---

In [308]:
# Construindo pipelines
pipe_lr    = Pipeline( [ ('scl', StandardScaler()), ('clf', LogisticRegression()) ] )
pipe_knn   = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])
pipe_dt    = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])
pipe_nb    = Pipeline([('scl', StandardScaler()), ('clf', GaussianNB())])
pipe_ridge = Pipeline([('scl', StandardScaler()), ('clf', RidgeClassifier())])
pipe_sgd = Pipeline([('scl', StandardScaler()), ('clf', SGDClassifier())])


# Lista de pipelines a serem executados
pipelines = [pipe_lr, pipe_knn, pipe_dt, pipe_nb, pipe_ridge, pipe_sgd]

# Dicionário para facilitar identificacao
pipe_dict = {0: 'Logistic Regression', 1: 'KNN', 2: 'Decision Tree', 3: 'Naive Bayes', 4: 'Ridge', 5: 'SGD'}

# aplicando fit
# Generaliza a execucao do fit de cada ultima funcao do pipe
for pipe in pipelines:
    pipe.fit(X_train, y_train.values.ravel())

# Compara acurácia
for i, pipe in enumerate(pipelines):
    print('%s pipeline test accuracy: %.3f' % (pipe_dict[i], pipe.score(X_test, y_test)))

# para cada modelo treinado obtem val score
best_acc = 0.0
best_clf = 0
best_pipe = ''
for i, pipe in enumerate(pipelines):
    # Descobre o melhor val.score e armazen em best_clf
    if pipe.score(X_test, y_test) > best_acc:
        best_acc = pipe.score(X_test, y_test)
        best_pipe = pipe
        best_clf = i
print('---\nClassifier with best accuracy: %s' % pipe_dict[best_clf])

Logistic Regression pipeline test accuracy: 0.909
KNN pipeline test accuracy: 0.903
Decision Tree pipeline test accuracy: 0.887
Naive Bayes pipeline test accuracy: 0.821
Ridge pipeline test accuracy: 0.911
SGD pipeline test accuracy: 0.903
---
Classifier with best accuracy: Ridge


In [309]:
# Salvando arquivo com o melhor modelo encontrado acima
joblib.dump(best_pipe, 'best_pipeline_senai.pkl', compress=1)

['best_pipeline_senai.pkl']

### Busca por hiperparametros usando GridSearchCV
---
A seguir será buscada, dentre todas as combinações definidas de hiperparametros, a combinação com o melhor score (e irá retornar esse valor).

In [310]:
# Set the parameters by cross-validation
tuned_parameters = [{'alpha': [1, 10, 100, 1000]},]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()
    
    clf = GridSearchCV(RidgeClassifier(), tuned_parameters, scoring='%s_macro' % score)
    clf.fit(X_train, y_train.values.ravel())

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("classification report:")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision



C:\Users\duanc\Miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\duanc\Miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\duanc\Miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\duanc\Miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: Undefi

Best parameters set found on development set:

{'alpha': 100}

Grid scores on development set:

0.799 (+/-0.101) for {'alpha': 1}
0.789 (+/-0.111) for {'alpha': 10}
0.921 (+/-0.066) for {'alpha': 100}
0.434 (+/-0.001) for {'alpha': 1000}

classification report:

              precision    recall  f1-score   support

           0       0.80      0.07      0.13        56
           1       0.89      1.00      0.94       441

    accuracy                           0.89       497
   macro avg       0.85      0.53      0.54       497
weighted avg       0.88      0.89      0.85       497


# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'alpha': 1}

Grid scores on development set:

0.615 (+/-0.050) for {'alpha': 1}
0.591 (+/-0.041) for {'alpha': 10}
0.528 (+/-0.026) for {'alpha': 100}
0.500 (+/-0.001) for {'alpha': 1000}

classification report:

              precision    recall  f1-score   support

           0       0.80      0.29      0.42        56
 

# Usando RandomizedSearchCV
---

Este método busca um conjunto aleatório de combinações possíveis de hiperparametros e retorna o modelo com o melhor score.

In [311]:
logistic = LogisticRegression(solver='saga', tol=0.01, max_iter=200, random_state=0)

distributions = dict(penalty=['l2', 'l1'])

clf = RandomizedSearchCV(logistic, distributions, random_state=0)

search = clf.fit(X_test, y_test.values.ravel())

search.best_params_

C:\Users\duanc\Miniconda3\lib\site-packages\sklearn\model_selection\_search.py:281: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


{'penalty': 'l2'}

# Validação Cruzada
---

A escolha de qual técnica de *Machine Learning* utilizar não é algo fixo e pode variar de problema para problema. Escolher a melhor técnica e a melhor separação treino/teste é algo bastante subjetivo e que demanda muitas iterações. 

No caso do uso da ferramenta ```train_test_split```, deseja-se que tanto o conjunto de dados de treino quanto o de teste (validação) sejam aproveitados ao máximo. Isso significa que cada instância da base de dados que é *utilizada no* ou *retirada do* conjunto de teste *veio* ou *vai*, necessariamente, para o conjunto de treino. Isso pode implicar num resultado não tão ótimo. 

A validação cruzada é a forma de testar diversos métodos num mesmo dataset de treino/teste. Além disso, pode-se utilizar várias combinações de treino/teste, a fim de garantir que o resultado não tenha viés. Ao final, obtem-se a média dos resultados e esse se torna a representação principal do modelo de predição.

### KFolds


Nesse tipo de classificação cruzada, primeiro os dados são divididos em k-subconjuntos treino/teste. O método irá, portanto, atribuir k-combinações diferentes para a separação treino/teste e, então, realizará treino e validação em cada uma das combinações. Ao final, os resultados serão uma média  de todos os k-resultados obtidos. A imagem abaixo ilustra o funcionamento de uma validação cruzada do tipo KFolds:

![cv_explained](cv_explained.png "Cross Validation Explained")

### Exemplificando KFold

In [312]:
# Amostra de dados
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])

# Montando Folds
kfold = KFold(5)

# Mostrando a divisão feita em cada KFold
for train, test in kfold.split(data):
    #print(train , test)
    print(data[train] , data[test])
    
# Exibindo a divisão com outro valor de k
kfold = KFold(8)
print("---")
for train, test in kfold.split(data):
    #print(train , test)
    print(data[train] , data[test])

[0.3 0.4 0.5 0.6 0.7 0.8] [0.1 0.2]
[0.1 0.2 0.5 0.6 0.7 0.8] [0.3 0.4]
[0.1 0.2 0.3 0.4 0.7 0.8] [0.5 0.6]
[0.1 0.2 0.3 0.4 0.5 0.6 0.8] [0.7]
[0.1 0.2 0.3 0.4 0.5 0.6 0.7] [0.8]
---
[0.2 0.3 0.4 0.5 0.6 0.7 0.8] [0.1]
[0.1 0.3 0.4 0.5 0.6 0.7 0.8] [0.2]
[0.1 0.2 0.4 0.5 0.6 0.7 0.8] [0.3]
[0.1 0.2 0.3 0.5 0.6 0.7 0.8] [0.4]
[0.1 0.2 0.3 0.4 0.6 0.7 0.8] [0.5]
[0.1 0.2 0.3 0.4 0.5 0.7 0.8] [0.6]
[0.1 0.2 0.3 0.4 0.5 0.6 0.8] [0.7]
[0.1 0.2 0.3 0.4 0.5 0.6 0.7] [0.8]


### Comparando a acurácia usando *Cross Validation* e somente Classificação

* **OBS:** ```cross_val_score``` aplica o método indicado como argumento em um número *k* de folds (especificado nos argumentos do método ```KFold()```) e obtém o score do fit de cada subconjunto. O método irá retornar os *scores* de cada iteração realizada por KFolds. A média desses valores será um resultado mais abrangente (próximo à realidade).

In [313]:
# SGD Classifier
sgd = SGDClassifier(random_state=42)
sgd.fit(X_train, y_train.values.ravel())
y_pred = sgd.predict(X_test)
print("Acuracia: %.2f%%" % (sgd.score(X_test, y_test)*100.0)) 

Acuracia: 88.73%


In [314]:
# Utilizando a validação cruzada do tipo KFold
kfold = KFold(n_splits=10)
model_kfold = SGDClassifier()
results_kfold = cross_val_score(model_kfold, X_train, y_train.values.ravel(), cv=kfold)

print("Scores: ", results_kfold) 

print("---\nAcuracia: %.2f%%" % (results_kfold.mean()*100.0)) 

Scores:  [0.84751773 0.12765957 0.85409253 0.89679715 0.11032028 0.86120996
 0.89323843 0.14590747 0.85409253 0.85765125]
---
Acuracia: 64.48%


## Stratified K-Fold

Neste método, cada conjunto comtém *aproximadamente* a mesma proporção de labels de destino que os dados completos. Isso é importante para que tanto no treino quanto no teste, o cenário seja o mais parecido com a realidade possível. 

Exemplo: considerando um dataset com transações de cartão de crédito em que se deseja predizer uma coluna "possivel_fraude", com valores 0 ou 1, suponha-se que haja 97% valores "zero" e 3% valores "um". Dessa forma, cada separação treino/teste será feita de forma que hajam apenas 3% de zeros no treino e no teste, aproximadamente, tornando mais parecido com a base de dados completa.

A seguir, o método de validação cruzada do tipo *Stratified K-Fold* é aplicado à base de dados do Senai (senai_inep2.csv).

In [315]:
skfold = StratifiedKFold(n_splits=10)
model_skfold = SGDClassifier()
results_skfold = cross_val_score(model_skfold, X_train, y_train.values.ravel(), cv=skfold)
print("Scores: ", results_skfold) 
print("Accuracy: %.2f%%" % (results_skfold.mean()*100.0))

Scores:  [0.79787234 0.75886525 0.8683274  0.8683274  0.8683274  0.78291815
 0.73309609 0.8683274  0.8683274  0.8683274 ]
Accuracy: 82.83%


### Leave One Out Cross-Validation (LOOCV)

Esse tipo de validação cruzada é semelhante ao KFolds, só que elevado ao seu máximo nível de *k*. Ou seja, a cada iteração, o fold terá tamanho 1: o teste será apenas 1 fold e os demais (k-1) será para treino. Será, então, executado treino e validação para cada uma das possibilidades. Ao final, assim como em KFolds, os resultados serão uma média de todos os k-resultados obtidos. 

**Observações:** Essa variação é útil quando os dados de treinamento são de tamanho limitado e o número de parâmetros a serem testados não é alto. Do contrário, vai exigir muito processamento e demandar muito tempo (principalmente para treino).

In [316]:
loocv = LeaveOneOut()
model_loocv = SGDClassifier()
results_loocv = cross_val_score(model_loocv, X_train, y_train.values.ravel(), cv=loocv)
print("Scores: ", results_loocv) 
print("Accuracy: %.2f%%" % (results_loocv.mean()*100.0))

Scores:  [1. 1. 1. ... 1. 1. 1.]
Accuracy: 76.53%


In [317]:
# A título de curiosidade: número de testes realizados
len(results_loocv)

2812

### Repetição Aleatória de divisão entre treino e teste (Repeated Random Test-Train Splits)

Essa técnica é uma mistura entre o tradicional ```train_test_split``` e o KFold (visto anteriormente). Primeiro, cria-se separações aleatórias de treino/teste dos dados, depois é feita sua avaliação (treino e validação). Esse mesmo processo de separação e execução dos dados de treino/teste é repetido multiplas vezes. No final, será retornado todos os valores de predição feito em cada *split* aleatório. A médias desses valores é utilizada para representar o resultado geral mais próximo à realidade.

In [318]:
kfold2 = ShuffleSplit(n_splits=10, test_size=0.30)
model_shufflecv = SGDClassifier()
results_4 = cross_val_score(model_shufflecv, X_train, y_train.values.ravel(), cv=kfold2)
print("Scores: ", results_4) 
print("Accuracy: %.2f%% (%.2f%%)" % (results_4.mean()*100.0, results_4.std()*100.0))

Scores:  [0.77132701 0.87559242 0.87085308 0.85900474 0.88270142 0.86848341
 0.86255924 0.29620853 0.76895735 0.88270142]
Accuracy: 79.38% (17.08%)


# Combinando Pipeline e Grid Search
---

In [319]:
#pipe_knn = Pipeline([('scl', StandardScaler()), ('clf', KNeighborsClassifier())])
pipetree = Pipeline([('scl', StandardScaler()), ('clf', DecisionTreeClassifier())])

#pipe = [pipe_knn, pipetre]
pipe = [pipetree]

param_range = [1, 2, 3, 4, 5]

# grid search params
#grid_params = [{'clf__criterion': ['gini', 'entropy'],
#               'clf__presort': [True, False]}]
grid_params = [{'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_leaf': param_range,
    'clf__max_depth': param_range,
    'clf__min_samples_split': param_range[1:]}] 

    #'clf__presort': [True, False]}]  PARAMETRO DEFASADO

# Construindo GridSearch
gs = GridSearchCV(estimator=pipetree,
                  param_grid=grid_params,
                  scoring='accuracy')

# Fit using grid search
gs.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.2f%%' % (gs.best_score_*100.0))

# Best params
print('\nBest params:\n', gs.best_params_)

Best accuracy: 91.39%

Best params:
 {'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__min_samples_leaf': 5, 'clf__min_samples_split': 2}


# Escolhendo Features 
---

* **Método *kbest***
    * Utiliza método *chi-quadrado* para escolher melhor conjunto de features de acordo com o seguinte critério:
        * Realiza um teste estatístico usando *chi-quadrado* entre cada features e classe
        * O teste do *chi-quadrado* “elimina” features com maior probabilidade de serem independentes da classe e, portanto, irrelevantes para a classificação.

In [320]:
print(X)
X_new = SelectKBest(chi2, k=3).fit_transform(X, y)
print(X_new)

      CO_IES  NO_IES  CO_CURSO  NO_CURSO  TP_TURNO  TP_GRAU_ACADEMICO  \
0       1400       1     20516         1         3                  1   
1       1400       1     20516         1         3                  1   
2       1400       1     20516         1         3                  1   
3       1400       1     20516         1         3                  1   
4       1400       1     20516         1         3                  1   
...      ...     ...       ...       ...       ...                ...   
3304   14786       5   1332810         5         3                  1   
3305   14786       5   1332810         5         3                  1   
3306   15445       0   1258151         0         3                  3   
3307   15445       0   1258151         0         3                  3   
3308   15445       0   1258151         0         3                  3   

      TP_MODALIDADE_ENSINO  ID_ALUNO  TP_COR_RACA  TP_SEXO  ...  \
0                        1      2519            1       

**Conforme visto acima, apenas os atributos ```CO_IES``` e ```CO_CURSO``` são os mais relevantes para predizer ```TP_SITUACAO```, segundo o resultado da aplicação do método *kBest*.**